# This notebook shows an example where a set of electrodes are selected from a dataset and then LFP is extracted from those electrodes and then written to a new NWB file


## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
    Note: currently this requires pynwb 1.3.3 or above and hdmf 2.0.1 or above, so use the latest development versions, not the conda versions
### 3) Install the ndx-fl-novela package. 
     conda install -c novelakrk ndx-fl-novela
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/to_be_updated

#### Make sure we're in the franklabnwb/franklabnwb directory. 
#### Eventually this will be unnecessary because the package will properly installed

#### Load all of the relevant modules and set the environment variables. 
Note that the datadir and datadir/analysis must exist

In [5]:
%env DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
%load_ext autoreload
%autoreload 2


import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj

# the commands below can be run once to update your global configuration

dj.config["enable_python_native_blobs"] = True
dj.config["database.user"] = 'root'
dj.config["databasse.password"] = 'tutorial'
dj.config.save_global()

import nwb_datajoint as nd
import ndx_franklab_novela.probe

data_dir = '/Users/loren/data/nwb_builder_test_data'
os.environ['NWB_DATAJOINT_BASE_DIR'] = data_dir
os.environ['KACHERY_STORAGE_DIR'] = os.path.join(data_dir, 'kachery-storage')



env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Next we select the NWB file, which corresponds to the dataset we want to extract LFP from

In [6]:
nwb_file_names = nd.common.Nwbfile().fetch('nwb_file_name')
# take the first one for this demonstration
nwb_file_name = nwb_file_names[0]
print(nwb_file_name)

beans20190718.nwb


#### Create the standard LFP Filters. This only needs to be done once.

In [7]:
nd.common.FirFilter().create_standard_filters()

### Select every other electrode for LFP

In [9]:
electrode_ids = nd.common.Electrode.fetch('electrode_id')
lfp_electrode_ids = electrode_ids[range(0, len(electrode_ids), 2)]
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, lfp_electrode_ids.tolist())


About to delete:
`common_ephys`.`l_f_p_selection__l_f_p_electrode`: 256 items
`common_ephys`.`l_f_p_selection`: 1 items


Proceed? [yes, No]:  yes


Committed.


### Or select one electrode for LFP


In [10]:
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, [2])


About to delete:
`common_ephys`.`__l_f_p_band`: 1 items
`common_ephys`.`l_f_p_band_selection__l_f_p_band_electrode`: 3 items
`common_ephys`.`l_f_p_band_selection`: 1 items
`common_ephys`.`_l_f_p`: 1 items
`common_ephys`.`l_f_p_selection__l_f_p_electrode`: 256 items
`common_ephys`.`l_f_p_selection`: 1 items


Proceed? [yes, No]:  yes


Committed.


### Populate the LFP table. Note that this takes 2 hours or so on a laptop if you use all electrodes

In [ ]:
import time
tic = time.perf_counter()
nd.common.LFP().populate()
toc = time.perf_counter()
toc - tic

writing new NWB file beans20190718_00000000.nwb
Output array should have shape (1091950, 128) and dtype <f8
Output array should have shape (212812, 128) and dtype <f8
Output array should have shape (2557744, 128) and dtype <f8
Output array should have shape (1356939, 128) and dtype <f8
Output array should have shape (1680945, 128) and dtype <f8
Checking output array shape is disabled, make sure portion of output array has shape (1091950, 128)
Checking output array shape is disabled, make sure portion of output array has shape (212812, 128)


### Now that we've created the LFP object we can perform a second level of filtering for a band of interest, in this case the theta band
We first need to create the filter

In [ ]:
lfp_sampling_rate = (nd.common.LFP() & {'nwb_file_name' : nwb_file_name}).fetch1('lfp_sampling_rate')
filter_name = 'Theta 5-11 Hz'
nd.common.FirFilter().add_filter(filter_name, lfp_sampling_rate, 'bandpass', [4, 5, 11, 12], 'theta filter for 1 Khz data')

Next we add an entry for the LFP Band and the electrodes we want to filter

In [ ]:
# assume that we've filtered all the electrodes; change this if not
lfp_band_electrode_ids = [1, 3, 5]

# set the interval list name corresponding to the second epoch (a run session)
interval_list_name = '02_r1'

# set the reference to -1 to indicate no reference for all channels
ref_elect = [-1]

# desired sampling rate
lfp_band_sampling_rate = lfp_sampling_rate // 10

In [ ]:
nd.common.LFPBandSelection().set_lfp_band_electrodes(nwb_file_name, lfp_band_electrode_ids, filter_name, interval_list_name, ref_elect, lfp_band_sampling_rate)

Check to make sure it worked

In [ ]:
nd.common.LFPBandSelection().LFPBandElectrode()

In [ ]:
nd.common.LFPBand().populate()

In [ ]:
nd.common.LFPBand()

### Now we can plot the original signal, the LFP filtered trace, and the theta filtered trace together.

In [26]:
import matplotlib.pyplot as plt

In [ ]:
rawdata = 